In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
df = pd.read_csv(r"C:\Users\furka\Desktop\ain4311\project\real-time-motion-detector\data\geophone-sensor-data.csv")

In [6]:
import sys
print(sys.executable)

C:\Users\furka\anaconda3\python.exe


In [7]:
df.head()

,timestamp,mean,top_3_mean,min,max,std_dev,median,q1,q3,skewness,dominant_freq,energy,activity,name
0,01:51:43,2046.726000,2057.666667,2035,2060,3.452186,2047.0,2043.0,2049.0,-0.152574,601.0,1.988682e+09,walking,Yusuf
1,01:51:46,2046.619333,2053.000000,2040,2053,3.580469,2047.0,2044.0,2050.0,-0.150615,300.0,1.988028e+09,walking,Yusuf
2,01:51:50,2046.598000,2056.000000,2030,2059,3.592826,2047.0,2044.0,2049.0,-0.370125,300.0,1.987897e+09,walking,Yusuf
3,01:51:53,2046.560000,2054.000000,2039,2055,3.414830,2047.0,2044.0,2049.0,-0.321296,300.0,1.987662e+09,walking,Yusuf
4,01:51:56,2046.656667,2055.000000,2034,2057,3.356802,2048.0,2044.0,2049.0,-0.379134,300.0,1.988255e+09,walking,Yusuf


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timestamp      1800 non-null   object 
 1   mean           1800 non-null   float64
 2   top_3_mean     1800 non-null   float64
 3   min            1800 non-null   int64  
 4   max            1800 non-null   int64  
 5   std_dev        1800 non-null   float64
 6   median         1800 non-null   float64
 7   q1             1800 non-null   float64
 8   q3             1800 non-null   float64
 9   skewness       1800 non-null   float64
 10  dominant_freq  1800 non-null   float64
 11  energy         1800 non-null   float64
 12  activity       1800 non-null   object 
 13  name           1800 non-null   object 
dtypes: float64(9), int64(2), object(3)
memory usage: 197.0+ KB


In [9]:
le = LabelEncoder()
scaler = MinMaxScaler()

In [10]:
# Split the data into time steps
def create_sequences(data, labels, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(labels[i+time_steps])
    return np.array(X), np.array(y)

In [16]:
def preprocess_data(df, target, time_steps):
    # Select numeric columns
    numeric_columns = df.select_dtypes(include=["float64", "int64"]).columns

    # Encode categorical columns
    if target in df.columns:
        df[target] = le.fit_transform(df[target])

    # Scale features
    scaled_features = scaler.fit_transform(df[numeric_columns])
    labels = df[target]

    # Create sequences
    X, y = create_sequences(scaled_features, labels, time_steps)
    return X, y

In [11]:
def train_lstm_model(df, target, timesteps):
    # Preprocess data
    X, y = preprocess_data(df, target, timesteps)

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # LSTM model
    lstm = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False),
        Dense(32, activation='relu'),
        Dense(len(np.unique(y)), activation='softmax')
    ])

    # Compile model
    lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    # Train model
    lstm.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

    # Evaluate model
    test_loss, test_accuracy = lstm.evaluate(X_test, y_test)
    y_pred = np.argmax(lstm.predict(X_test), axis=1)

    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    
    save_model("lstm_geophone_sensor_name_predictv2", lstm)
    
    return {
        "summary": lstm.summary(),
        'model': lstm,
        'classification_report': classification_report(y_test, y_pred, output_dict=True),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'test_acc': test_accuracy
    }

In [12]:
def save_model(name, model):

    return print("atlattık")

In [13]:
def main():
    # Train and evaluate model
    # Load dataset
    df = pd.read_csv("../data/geophone-sensor-data.csv")

    # Train and save model
    results = train_lstm_model(df, target="name", timesteps=3)

    print("Model training complete. Saved to models directory.")
    
    # Print results
    print("Model Performance:")
    print("-------------------")
    print("LSTM Classification Report:")
    print(results['classification_report'])
    print(results['test_acc'])

In [61]:
if __name__ == "__main__":
    main()

Epoch 1/50


C:\Users\furka\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2334 - loss: 1.5153 - val_accuracy: 0.4667 - val_loss: 1.2772
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4834 - loss: 1.2222 - val_accuracy: 0.6389 - val_loss: 1.1205
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6288 - loss: 1.0146 - val_accuracy: 0.6361 - val_loss: 0.9409
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6178 - loss: 0.9111 - val_accuracy: 0.6500 - val_loss: 0.8707
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6395 - loss: 0.8651 - val_accuracy: 0.6417 - val_loss: 0.8303
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6550 - loss: 0.8275 - val_accuracy: 0.6750 - val_loss: 0.7942
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6500 - loss: 0.8023 - val_accuracy: 0.6444 - val_loss: 0.7902
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6237 - loss: 0.8131 - val_accuracy: 0.6667 - val_loss: 0.7711
Epo

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        72
           1       0.97      0.91      0.94        80
           2       0.57      0.78      0.66        59
           3       0.81      0.78      0.79        77
           4       0.73      0.56      0.63        72

    accuracy                           0.81       360
   macro avg       0.81      0.81      0.80       360
weighted avg       0.82      0.81      0.81       360

Confusion Matrix:
[[72  0  0  0  0]
 [ 3 73  2  1  1]
 [ 0  0 46  3 10]
 [ 0  0 13 60  4]
 [ 0  2 20 10 40]]
atlattık


Model: "sequential_9"

...

 Total params: 65,105 (254.32 KB)

 Trainable params: 21,701 (84.77 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 43,404 (169.55 KB)

Model training complete. Saved to models directory.
Model Performance:
-------------------
LSTM Classification Report:
{'0': {'precision': 0.96, 'recall': 1.0, 'f1-score': 0.9795918367346939, 'support': 72}, '1': {'precision': 0.9733333333333334, 'recall': 0.9125, 'f1-score': 0.9419354838709678, 'support': 80}, '2': {'precision': 0.5679012345679012, 'recall': 0.7796610169491526, 'f1-score': 0.6571428571428571, 'support': 59}, '3': {'precision': 0.8108108108108109, 'recall': 0.7792207792207793, 'f1-score': 0.794701986754967, 'support': 77}, '4': {'precision': 0.7272727272727273, 'recall': 0.5555555555555556, 'f1-score': 0.6299212598425198, 'support': 72}, 'accuracy': 0.8083333333333333, 'macro avg': {'precision': 0.8078636211969545, 'recall': 0.8053874703450974, 'f1-score': 0.8006586848692011, 'support': 360}, 'weighted avg': {'precision': 0.8202469675062269, 'recall': 0.8083333333333333, 'f1-score': 0.8088979533744385, 'support': 360}}
0.8083333373069763


In [14]:
from tensorflow.keras.models import load_model

model_directory = r'C:\Users\furka\Desktop\ain4311\project\real-time-motion-detector\models'
model_path = os.path.join(model_directory, '3', 'lstm_geophone_sensor_name_predictv2.h5')
model = load_model(model_path)

model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 64)             │        19,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,703 (84.78 KB)

 Trainable params: 21,701 (84.77 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)